In [1]:
!pip install mteb

In [3]:
import torch
from transformers import AutoTokenizer, AutoModel
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Using device:", device)

Using device: mps


In [4]:
ls trained

e5-base-v2-100k/     e5-base-v2-50k/      e5-base-v2-esp-150k/


In [15]:
# load from disk
class embedding_model:
    def __init__(self, model, tokenizer, device, inference=False):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.device = device
        if inference:
            self.model.eval()
            self.model.requires_grad_(False)

    def average_pool(last_hidden_states: torch.Tensor,
                 attention_mask: torch.Tensor) -> torch.Tensor:
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
    
    def encode(self, sentences, batch_size=32):
        embeddings = []
        for i in range(0, len(sentences), batch_size):
            batch_sentences = sentences[i:i+batch_size]
            for sentence in batch_sentences:
                embedding = self.__call__(sentence)
                # Take the first element of the tensor to get a 1D tensor
                embedding = embedding[0]
                embeddings.append(embedding.cpu().numpy())
        return embeddings

    def __call__(self, data):
        tokens_and_mask = self.tokenizer(data, return_tensors='pt', padding=True, truncation=True, max_length=512).to(self.device)
        model_output = self.model(tokens_and_mask["input_ids"], attention_mask=tokens_and_mask["attention_mask"])
        embedding = embedding_model.average_pool(model_output.last_hidden_state, attention_mask=tokens_and_mask["attention_mask"])
        # normalize the embedding
        embedding = embedding / embedding.norm(dim=-1, keepdim=True)
        return embedding    

dyn = embedding_model(AutoModel.from_pretrained("/Users/willhathaway/classes/6.s898/mulitlingual-embeddings/webteam_work/trained/e5-base-v2-esp-150k"), AutoTokenizer.from_pretrained("tokenizers/e5-base-v2"), device=device, inference=True)
# sta = embedding_model(AutoModel.from_pretrained("models/e5-base-v2"), AutoTokenizer.from_pretrained("tokenizers/e5-base-v2"), device=device, inference=True)
# mult = embedding_model(AutoModel.from_pretrained("models/multilingual-e5-small"), AutoTokenizer.from_pretrained("tokenizers/multilingual-e5-small"), device=device, inference=True)



# put into dictionary
models = {
    "dyn": dyn,
}

## MTEB

In [2]:
from mteb import MTEB
from sentence_transformers import SentenceTransformer

model_name = "dyn"

# model = SentenceTransformer(model_name)
evaluation = MTEB(tasks=["Banking77Classification"], task_langs=["en", "de"])
results = evaluation.run(models[model_name], output_folder=f"results/{model_name}")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


: 

In [18]:
results

{}

In [21]:
results_sta

{'Banking77Classification': {'mteb_version': '1.1.1',
  'dataset_revision': '0fd18e25b25c072e09e0d92ab615fda904d66300',
  'mteb_dataset_name': 'Banking77Classification',
  'test': {'accuracy': 0.7663636363636364,
   'f1': 0.7571862523229754,
   'accuracy_stderr': 0.008049329276696824,
   'f1_stderr': 0.009676908587947413,
   'main_score': 0.7663636363636364,
   'evaluation_time': 242.59}}}